In [ ]:
# Importing the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import matplotlib
np.__version__, pd.__version__, sns.__version__, matplotlib.__version__

**1. Load Data**

In [ ]:
# Load the data as pandas dataframe
df = pd.read_csv('../../dataset/cars.csv')

In [ ]:
# Check the first five rows in the data
df.head()

In [ ]:
# Check the number of rows in the data
df.shape

In [ ]:
# Show the non null values and data type of each column
df.info()

In [ ]:
# List out the column names
df.columns

As the name of columns are perfect to be used, no updates are applied to make them more readable and accessible.

**2 Explatory Data Analysis**

2.1 Cleaning the data

First all the unnecessary feature data are to be removed such as units and brand related information.

In [ ]:
# Here we define a function to return float values for features with pattern "floatvalue + unit"
# Example for feature km_driven, if data is "12345 km", then 12345.00 is returned

def getFloatValues(featureValues):
    # all values are converted to string in case there are any float or integer values
    featureValues = featureValues.astype(str)

    # the first part of values are separated and converted to float values and mapped
    # in case the values can not be converted to float, then values are set to 0
    for index, x in enumerate(featureValues):
        try:
            featureValues[index] = float(x.split(' ')[0])
        except ValueError:
            featureValues[index] = 0

    return featureValues

In [ ]:
# For feature name, the brand name of the car is kept. The first word in the name is assumed to be brand name
df['name'] = df['name'].map(lambda x : x.split(' ')[0])

# For feature mileage, the unit kmpl is removed and values converted into float values
df['mileage'] = getFloatValues(df['mileage'])
df['mileage'] = df['mileage'].astype('float')

# For feature engine, the unit CC is removed and values converted into float values
df['engine'] = getFloatValues(df['engine'])
df['engine'] = df['engine'].astype('float')

# For feature max_power, the unit bhp is removed and values converted into float values
df['max_power'] = getFloatValues(df['max_power'])
df['max_power'] = df['max_power'].astype('float')

# For feature torque, it is dropped due insignifcance to car company
df = df.drop('torque', axis = 1)

# For feature fuel, all the rows with values LPG and CNG are removed
df = df[~df['fuel'].isin(['CNG', 'LPG'])]

df.head()

In [ ]:
df.shape

In [ ]:
df.info()

2.2 Univariate analysis

Countplot

Count plot can be used to see the number of rows for a label for a categorical feature

In [ ]:
# Let's see how many individual and dealer sellers are there
sns.countplot(data = df, x = 'seller_type')

Distribution plot

Distribution plot can plot the distribution of continous values. It helps in identifying the type of distribution for the feature

In [ ]:
# Distribution plot for selling prices
sns.displot(data = df, x = 'selling_price')

2.2 Multivariate Analysis

Multiple variable exploratory analysis

Boxplot

In [ ]:
# Box plot for 'owner' and 'selling_price'
sns.boxplot(x = df["owner"], y = df["selling_price"]);
plt.ylabel("Selling Price")
plt.xlabel("Owner")

Scatterplot

In [ ]:
# Scatter plot for mileage and selling price with respect to fuel type

sns.scatterplot(x = df['mileage'], y = df['selling_price'], hue =df['fuel'])

Corelation Matrix

In [ ]:
# Let's check out heatmap

plt.figure(figsize = (15, 8))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm")

Currently, feature max_power and engine have shown strong correlation to selling price. However, the above graph does not include categorical features.

Label Encoding

Lets encode the labels for the present categorical featues

In [ ]:
# Importing the LabelEncoder
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

All the categorical features except 'owner' are label encoded through LabelEncoder class

In [ ]:
# Loading the features into list
categorical_features = ['name', 'seller_type', 'fuel', 'transmission']

# Each feature in the list are label encoded 
for feature in categorical_features:
    df[feature] = le.fit_transform(df[feature])
    le.transform(le.classes_)

As the feature 'owner' has the requirement to assign custom encoding to each label, each label has mapped to the required encoding value.

In [ ]:
mapping = {
    'First Owner': 1,
    'Second Owner': 2,
    'Third Owner': 3,
    'Fourth & Above Owner': 4,
    'Test Drive Car': 5
}

df['owner'] = df['owner'].map(lambda x : mapping[x])

# Removing the rows with Test Drive Car value
df = df[~df['owner'].isin([5])]

Now that all the categorical labels have been encoded into integer values, lets look into our current data

In [ ]:
df.head()

Now the correlation matrix will display the values for these converted features as well

In [ ]:
plt.figure(figsize = (15, 8))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm")

The most correlated features are still found to be engine and max_power

**Predictive Power Socre**

Let's check the predictive power scores of features. This graph plots the direct predictive power of a feature against another feature.

In [ ]:
import ppscore as pps

dfcopy = df.copy()

matrix_df = pps.matrix(dfcopy)[['x', 'y', 'ppscore']].pivot(columns='x', index='y', values='ppscore')

#plot
plt.figure(figsize = (15,8))
sns.heatmap(matrix_df, vmin=0, vmax=1, cmap="Blues", linewidths=0.5, annot=True)

**3. Feature Selection**

In [ ]:
# According the corelation matrix and pps scores, the most strong features are max_power, engine and mileage
# Therefore, X is set to those features

X = df[['max_power', 'engine', 'mileage']]

# y is the selling price. As selling price values are too big, they will transformed with log
y = np.log(df['selling_price'])

In [ ]:
print(y)

**4. Test Train Split**

The training and test data are split into 7:3 ratio

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

**5 Preprocessing**

5.1 Checking for null values

Here all the null values for the features are filled with appropriate values

In [ ]:
# Checking for null values in X for training set
X_train[['max_power', 'engine', 'mileage']].isna().sum()

In [ ]:
# Checking for null values in X for test set
X_test[['max_power', 'engine', 'mileage']].isna().sum()

In [ ]:
#Checking for null values in y
y_train.isna().sum()
y_test.isna().sum()

In [ ]:
# Distribution plot for max_power
sns.displot(data=df, x = 'max_power')

In [ ]:
# Distribution plot for mileage
sns.displot(data=df, x = 'engine')

In [ ]:
# Distribution plot for mileage
sns.displot(data=df, x = 'mileage')

In [ ]:
# Distribution plot selling price for training
sns.displot(y_train)

In [ ]:
# The null values for features max power and engine are filled with median values
X_train['max_power'].fillna(X_train['max_power'].median(), inplace=True)
X_train['engine'].fillna(X_train['engine'].median(), inplace=True)

# The null values for feature mileage are filled with mean values as mileage values resemnle normal distribution
X_train['mileage'].fillna(X_train['mileage'].mean(), inplace= True)

In [ ]:
# The same process above is repeated for test data as well
X_test['max_power'].fillna(X_test['max_power'].median(), inplace=True)
X_test['engine'].fillna(X_train['engine'].median(), inplace=True)
X_test['mileage'].fillna(X_train['mileage'].mean(), inplace= True)

In [ ]:
# Final check for null values for X in training and test sets
X_train[['max_power', 'engine', 'mileage']].isna().sum()
X_test[['max_power', 'engine', 'mileage']].isna().sum()

As the values for y have no null values, the y_train and y_test are not updated in this process.

**5.2 Checking Outliers**

In [ ]:
# Create a dictionary of columns

col_dict = {'max_power': 1, 'engine': 2, 'mileage': 3}

# Box plots to detect outliers in each variables

for variable, i in col_dict.items():
  plt.subplot(5,4,i)
  plt.boxplot(X_train[variable])
  plt.title(variable)

plt.show()

In [ ]:
# This method takes feature name and training set as parameters and print the number and percentage of outliers present for the feature
def outlier_count(col, data = X_train):

    # calculate your 25% quatile and 75% quartile
    q75, q25 = np.percentile(data[col], [75, 25])

    # calculate your inter quartile
    iqr = q75 - q25

    # Calculating max_value and min_value for the feature
    min_val = q25 - (iqr*1.5)
    max_val = q75 + (iqr*1.5)

    # The number of outliers is counted on basis if the value is greater than max value or less than the min value
    outlier_count = len(np.where((data[col] > max_val) | (data[col] < min_val))[0])

    # calculate the percentage of the outliers
    outlier_percent = round(outlier_count/len(data[col])*100, 2)

    if(outlier_count > 0):
        print("\n"+15*'-' + col + 15*'-'+"\n")
        print('Number of outliers: {}'.format(outlier_count))
        print('Percent of data that is outlier: {}%'.format(outlier_percent))

In [ ]:
# Method call for finding out outliers in all features in training set
for col in X_train.columns:
    outlier_count(col)

From above, feature engine has the highest number of outliers at 14.88%. While the percentage is high and may affect the training model but it has shown good scores in corelation matrix and pps score graph. Usually, the values of engine do tend to affect the prices in real world as well. Hence, engine will be used to train the model.

To mitigate the effects of outliers, all the training set will be scaled by Standard Scaler to covert the data into more consistent form.

**5.3 Scaling**

For scaling the features of training and test set, Robust Scaler will be used. The process of scaling generally helps in faster convergence while training data.

In [ ]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()

X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

In [ ]:
# Shape check for X_train, X_test, y_train, y_test before model fitting
print("Shape of X_train: ", X_train.shape)
print("Shape of X_test: ", X_test.shape)
print("Shape of y_train: ", y_train.shape)
print("Shape of y_test: ", y_test.shape)

In [ ]:
intercept = np.ones((X_train.shape[0], 1))
X_train   = np.concatenate((intercept, X_train), axis=1)
intercept = np.ones((X_test.shape[0], 1))
X_test    = np.concatenate((intercept, X_test), axis=1)

**6. Modeling**

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
import math
import mlflow


class LinearRegression(object):
    
    #in this class, we add cross validation as well for some spicy code....
    kfold = KFold(n_splits=5)
            
    def __init__(self, regularization, lr, method, theta_type, momentum, num_epochs=500, batch_size=50, cv=kfold):
        self.lr         = lr
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.method     = method
        self.theta_type = theta_type
        self.momentum = momentum
        self.cv         = cv
        self.regularization = regularization

    def mse(self, ytrue, ypred):
        return ((ypred - ytrue) ** 2).sum() / ytrue.shape[0]
    
    def r2(self, ytrue, ypred):
        return r2_score(ytrue, ypred)
    
    def getXavierTheta(self, num):
        lower, upper = -(1.0 / math.sqrt(num)), (1.0 / math.sqrt(num))

        numbers = np.random.rand(num)

        scaled = lower + numbers * (upper - lower)

        return scaled
    
    def fit(self, X_train, y_train):
            
        #create a list of kfold scores
        self.kfold_scores = list()
        
        #reset val loss
        self.val_loss_old = np.infty

        #kfold.split in the sklearn.....
        #5 splits
        for fold, (train_idx, val_idx) in enumerate(self.cv.split(X_train)):
            
            X_cross_train = X_train[train_idx]
            y_cross_train = y_train[train_idx]
            X_cross_val   = X_train[val_idx]
            y_cross_val   = y_train[val_idx]

            self.theta = self.getXavierTheta(X_cross_train.shape[1]) if self.theta_type == 'xavier' else np.zeros(X_cross_train.shape[1])
            
            # self.theta = np.zeros(X_cross_train.shape[1])
            
            #define X_cross_train as only a subset of the data
            #how big is this subset?  => mini-batch size ==> 50
            
            #one epoch will exhaust the WHOLE training set
            with mlflow.start_run(run_name=f"Fold-{fold}", nested=True):
                
                params = {"method": self.method, "lr": self.lr, "reg": type(self).__name__}
                mlflow.log_params(params=params)
                
                for epoch in range(self.num_epochs):
                
                    #with replacement or no replacement
                    #with replacement means just randomize
                    #with no replacement means 0:50, 51:100, 101:150, ......300:323
                    #shuffle your index
                    perm = np.random.permutation(X_cross_train.shape[0])
                            
                    X_cross_train = X_cross_train[perm]
                    y_cross_train = y_cross_train[perm]
                    
                    if self.method == 'stochastic':
                        for batch_idx in range(X_cross_train.shape[0]):
                            X_method_train = X_cross_train[batch_idx].reshape(1, -1) #(11,) ==> (1, 11) ==> (m, n)
                            y_method_train = y_cross_train[batch_idx] 
                            train_loss = self._train(X_method_train, y_method_train)
                    elif self.method == 'mini-batch':
                        for batch_idx in range(0, X_cross_train.shape[0], self.batch_size):
                            #batch_idx = 0, 50, 100, 150
                            X_method_train = X_cross_train[batch_idx:batch_idx+self.batch_size, :]
                            y_method_train = y_cross_train[batch_idx:batch_idx+self.batch_size]
                            train_loss = self._train(X_method_train, y_method_train)
                    else:
                        X_method_train = X_cross_train
                        y_method_train = y_cross_train
                        train_loss = self._train(X_method_train, y_method_train)

                    mlflow.log_metric(key="train_loss", value=train_loss, step=epoch)

                    yhat_val = self.predict(X_cross_val)
                    val_loss_new = self.mse(y_cross_val, yhat_val)
                    mlflow.log_metric(key="val_loss", value=val_loss_new, step=epoch)
                    
                    #record dataset
                    mlflow_train_data = mlflow.data.from_numpy(features=X_method_train, targets=y_method_train)
                    mlflow.log_input(mlflow_train_data, context="training")
                    
                    mlflow_val_data = mlflow.data.from_numpy(features=X_cross_val, targets=y_cross_val)
                    mlflow.log_input(mlflow_val_data, context="validation")
                    
                    #early stopping
                    if np.allclose(val_loss_new, self.val_loss_old):
                        break
                    self.val_loss_old = val_loss_new
            
                self.kfold_scores.append(val_loss_new)
                print(f"Fold {fold}: {val_loss_new}")
            
                    
    def _train(self, X, y):
        yhat = self.predict(X)
        m    = X.shape[0]        
        grad = (1/m) * X.T @(yhat - y) + self.regularization.derivation(self.theta)
        self.theta = self.theta - self.lr * grad
        return self.mse(y, yhat)
    
    def predict(self, X):
        return X @ self.theta  #===>(m, n) @ (n, )
    
    def _coef(self):
        return self.theta[1:]  #remind that theta is (w0, w1, w2, w3, w4.....wn)
                               #w0 is the bias or the intercept
                               #w1....wn are the weights / coefficients / theta
    def _bias(self):
        return self.theta[0]

Now we can create `Ridge`, `Lasso` and `Elastic` class that extends the `LinearRegression`, with added penalty.

In [ ]:
class LassoPenalty:
    
    def __init__(self, l):
        self.l = l # lambda value
        
    def __call__(self, theta): #__call__ allows us to call class as method
        return self.l * np.sum(np.abs(theta))
        
    def derivation(self, theta):
        return self.l * np.sign(theta)
    
class RidgePenalty:
    
    def __init__(self, l):
        self.l = l
        
    def __call__(self, theta): #__call__ allows us to call class as method
        return self.l * np.sum(np.square(theta))
        
    def derivation(self, theta):
        return self.l * 2 * theta
    
class ElasticPenalty:
    
    def __init__(self, l = 0.1, l_ratio = 0.5):
        self.l = l 
        self.l_ratio = l_ratio

    def __call__(self, theta):  #__call__ allows us to call class as method
        l1_contribution = self.l_ratio * self.l * np.sum(np.abs(theta))
        l2_contribution = (1 - self.l_ratio) * self.l * 0.5 * np.sum(np.square(theta))
        return (l1_contribution + l2_contribution)

    def derivation(self, theta):
        l1_derivation = self.l * self.l_ratio * np.sign(theta)
        l2_derivation = self.l * (1 - self.l_ratio) * theta
        return (l1_derivation + l2_derivation)
    
class Lasso(LinearRegression):
    
    def __init__(self, method, lr, theta, momentum, l):
        self.regularization = LassoPenalty(l)
        super().__init__(self.regularization, lr, method, theta, momentum)
        
class Ridge(LinearRegression):
    
    def __init__(self, method, lr, theta, momentum, l):
        self.regularization = RidgePenalty(l)
        super().__init__(self.regularization, lr, method, theta, momentum)
        
class ElasticNet(LinearRegression):
    
    def __init__(self, method, lr, l, theta, momentum, l_ratio=0.5):
        self.regularization = ElasticPenalty(l, l_ratio)
        super().__init__(self.regularization, lr, method, theta, momentum)


**Experiment**

In [ ]:
#helper function for looping classnames
import sys

def str_to_class(classname):
    return getattr(sys.modules[__name__], classname)

In [ ]:
import itertools

regs = ["Ridge", "Lasso", "ElasticNet"]
momentums = [False, True]
methods = ["batch"]
thetas = ['zero', 'xavier']
learning_rates = [0.01, 0.001, 0.0001]

all_combinations = list(itertools.product(regs, momentums, methods, thetas, learning_rates))

parameters = []

for combo in all_combinations:
    parameters.append({
        "reg": combo[0],
        "momentum": combo[1],
        "method": combo[2],
        "theta": combo[3],
        "lr": combo[4],
        "l": 0.1
    })

In [ ]:
for params in parameters:
    run_name=f"method-{params['method']}-lr-{params['lr']}-reg-{params['reg']}-theta-{params['theta']}-momentum-{params['momentum']}"
    print(run_name)
    mlflow.start_run(run_name=f"method-{params['method']}-lr-{params['lr']}-reg-{params['reg']}-theta-{params['theta']}-momentum-{params['momentum']}", nested=True)

    type_of_regression = str_to_class(params['reg'])    #Ridge, Lasso, ElasticNet
    del params["reg"]
    model = type_of_regression(**params)  
    model.fit(X_train, y_train.values)
    yhat = model.predict(X_test)
    mse  = model.mse(yhat, y_test)
    r2 = model.r2(yhat, y_test)

    mlflow.log_metric(key="test_mse", value=mse)
    mlflow.log_metric(key="r2", value=r2)

    signature = mlflow.models.infer_signature(X_train, model.predict(X_train))
    mlflow.sklearn.log_model(model, artifact_path='model', signature=signature)

    mlflow.end_run()

In [ ]:


print(len(parameters))

In [ ]:
*****************

**7. Testing**

As the best MSE value has been satisfactory to our need, the model is tested on test set below. The X_test has been already scaled and y_test has been logarithm scaled like the training set.

In [ ]:
from sklearn.metrics import mean_squared_error

yhat = grid.predict(X_test)

mean_squared_error(yhat, y_test)

The mean squared error from the test set is similar to best mse from the training set. Hence, the model has been successfully trained. 

**8. Feature Analysis**

The main point of feature analysis is to distinguish the most important and relevant features in the model. Each feature may provide different level of significance in the model.

8.1. Using the available grid methods

The grid also provides the level of importance of each features used in training the model along with the best parameters.

Each features is given a score from 0 t0 1 on their importance in training the data. The values are plotted in bar graph below


In [ ]:
xgb_best_estimator = grid.best_estimator_

# Extracting the feature importance scores from the grid
xgb_best_estimator.feature_importances_

# Bar plot for the features and thier importance
plt.barh(X.columns, xgb_best_estimator.feature_importances_)
plt.xlabel("XGB Regressor Feature Importance")

8.2. Using SHAP

The SHAP libaray is a model agnostic library. It calculates shap values which considers all possible combinations of features and their contributions to the prediction.

In [ ]:
import shap

explainer = shap.TreeExplainer(xgb_best_estimator)
shap_values = explainer.shap_values(X_test)
shap.summary_plot(shap_values, X_test, plot_type="bar", feature_names = X.columns)

From above analysis, feature max_power has the most siginifance in predicting the selling price. Both features mileage and engine have significantly less affect in prediction compared to max_power. 

**9. Inference**

As our model has been trained to fit our needs, the model will exported to model file using the pickle library. The model will be accesible to the app to be imported and predict the selling price based on user inputs. 

In [ ]:
# Importing pickle library
import pickle

# Exporting the model to selling-price.model
filename = '../model/selling-price.model'
pickle.dump(grid, open(filename, 'wb'))

In [ ]:
# We will also dump the scaler values for future use
scaler_filename = '../model/scaler.pkl'
pickle.dump(scaler, open(scaler_filename, 'wb'))

To use the model, it can be imported and be provided with the feature values as below

In [ ]:
# Importing the model
selling_price_model = pickle.load(open("../model/selling-price.model", "rb"))

# Creating a dummy sample
sample = {
    "max_power": [100],
    "engine": [1200],
    "mileage": [23]
}

# Convert the sample to panda dataframe
sample = pd.DataFrame(sample)

# Scale the sample using the same scaler used for X_train and X_set
scaled_sample = scaler.transform(sample)

# Use the model to predict the selling price
predicted_selling_price = selling_price_model.predict(scaled_sample)

# As the we have log transformed the y while training and set, we will need to exponent transform the predicted value for correct prediction
predicted_selling_price = np.exp(predicted_selling_price)

print("The predicted selling price is " + str(predicted_selling_price[0]))

**10. Report**

10.1. Summary

The above implemenation has resulted in creating a model to meet the requirements of the car company. The prediction result from the model have been found to be good. The major blocks of the above implemenation are:

- Number of final samples used: 8033
- Split ratio for training and test: 7:3
- Features chosen for training model: max_power, mileage, engine
- Feature to be predicted: selling_price
- Scaler Used: Robust Scaler
- Algorithm Chosen to train the model: XGBRegressor
- Best MSE result for the test set: -0.087 (Negative Squared Mean Error)


10.2. Features Discussion

There were 12 features to be considered for training our model. The feature torque was dropped earlier on due to insigifance and lack of knowledge to the car company. The categorical features were label encoded and the float values were extracted for the other coninous features to be plotted in the correlation matrix and PPS score graph.

The correlation matrix showed strong values for year, engine and max_power. The PPS score graph showed strong values for engine, mileage and max_power. The features for training were selected the based on PPS score graph, and the general knowledge that these feature do signifacntly affect the prices of car in the real world.

In the feature analysis section, the feature max_power has been shown to have significant importance while predicting the result. The feature engine and mileage have comparably equal importance in prediction. Hence, these features have been appropriate for training the model.


12.3. Preprocessing Discussion

The feature engine had significantly more outliers than the other two features. A high number of outliers may affect the training and prediction result of the model. However, the feature was deemed important in order to be dropped. The scaling process was relied upon to mitigate the effects of any outliers on the training. The following three scalers were considered:

- Standard Scaler
- Min Max Scaler
- Robust Scaler

From the above list, Min Max Scaler was found to be too sensitive to outliers. Standard Scaler could perform well but it was too sensitive to extreme outliers. Hence, Robust Scaler was used which was found to be less sensitive to extreme outliers.


10.4 Algorithm Selection Discussion

In order to select the appropriate model for training, the cross validation method was used to get the best mean squared error among different algoriths. The list of the algorithms and their mean scores from cross validation methods are as follows:

| Algorthm | Mean from Cross Validaton |
| -------- | ------------------------- |
| Linear Regression | -0.2812275460816783 |
| SVR | -0.22900504420396048 |
| KNeighbors Regressor | -0.10405329584844414 |
| Decision-Tree Regressor | -0.09731487031196318 |
| Random-Forest Regressor | -0.08972391099371599 |
| XGBregressor |-0.08678452365550038 |

*Scores are from 29th August 2023. Scores are subjected to change if the cross validation is run again*

As from the above table, the best values were acheived from Random Foreset Regressor and XGBRegressor with latter scoring better with some magnitude. Both algorithm were trained and tested using equivalent paramters and XBG was found to be produce consistent better results. Hence, XBG Regressor was chose for the final implmentation.

XGB Regressor has been found to be more strong boosting and strong predictive algorithm which could handle our dataset. The parameters for the XBG Regressor were set as follows:

- max_depth: [5, 10, None],
- n_estimators: [100, 200, 300, 400, 500],
- learning_rate: [0.1, 0.2]

Grid Search was applied to find the best parameter values and best mse score from the algorithm that would train our model. Following were results:

Best value for: 
- learning_rate: 0.1
- max_depth: None
- n_estimators: 500

Best MSE from grid: -0.08626911482197705


10.5 Testing and Conclusion

As the final step in our implementaion, the model was tested using the split test. The MSE for the test was found to be **0.08371498311349586**

The training and testing of model was successful due to favorable results achieved above.